In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
device = torch.device("mps")
print(f"Using {device} device")

Using mps device


In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [5]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([8], device='mps:0')


In [7]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [8]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [9]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [11]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.3257,  0.7162,  0.1434,  0.1634, -0.3598, -0.1695, -0.1532,  0.0739,
         -0.0709,  0.4122, -0.0616,  0.2857,  0.1809,  0.0058,  0.2580,  0.5221,
          0.5822,  0.1242,  0.1036,  0.1829],
        [ 0.1483,  0.8140, -0.0098,  0.1867, -0.2701,  0.1085,  0.1745,  0.2713,
         -0.0089,  0.1973, -0.1292,  0.2576,  0.3136,  0.1250,  0.6764,  0.6818,
          0.4515,  0.2926,  0.3984,  0.0048],
        [ 0.2451,  0.7283,  0.2853,  0.2432, -0.2842, -0.3214, -0.1836,  0.0514,
          0.3117,  0.1579, -0.2123,  0.4041,  0.0541,  0.2206,  0.6816,  0.5178,
          0.3341,  0.4899,  0.0573,  0.2588]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.3257, 0.7162, 0.1434, 0.1634, 0.0000, 0.0000, 0.0000, 0.0739, 0.0000,
         0.4122, 0.0000, 0.2857, 0.1809, 0.0058, 0.2580, 0.5221, 0.5822, 0.1242,
         0.1036, 0.1829],
        [0.1483, 0.8140, 0.0000, 0.1867, 0.0000, 0.1085, 0.1745, 0.2713, 0.0000,
         0.1973, 0.0000, 0.2576, 0.3136, 0.1250, 0.67

In [12]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [13]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [14]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0081, -0.0271, -0.0179,  ..., -0.0179,  0.0318,  0.0164],
        [ 0.0294, -0.0013,  0.0014,  ..., -0.0245,  0.0188,  0.0132]],
       device='mps:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([ 0.0301, -0.0236], device='mps:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0392, -0.0409,  0.0126,  ...,  0.0385, -0.0320,  0.0156],
        [ 0.0361,  0.0379,  0.0364,  ..., -0.0298, -0.0352, -0.0099]],
       device='mps:0', grad_fn=<Slice